In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, TensorDataset

from tensorflow.keras import datasets

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device using: {device}")

Device using: cpu


In [3]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [4]:
x_train = torch.tensor(x_train / 255.0, dtype=torch.float32)
x_test = torch.tensor(x_test / 255.0, dtype=torch.float32)

In [5]:
y_train = torch.tensor(y_train, dtype=torch.long).squeeze()
y_test = torch.tensor(y_test, dtype=torch.long).squeeze()

In [6]:
def one_hot_encoded(tensor_array, label_num=10):
    torch_tensor = torch.zeros(tensor_array.size(0),label_num)
    torch_tensor[[i for i in range(len(tensor_array))], tensor_array] = 1
    return torch_tensor

In [7]:
train_dataset=TensorDataset(x_train.reshape(-1,3,32,32),one_hot_encoded(y_train))
train_data_loader=DataLoader(train_dataset , batch_size=10, shuffle=True)

for data, target in train_data_loader:
    print("Batch shape:", data.shape, target.shape)

    # To check sample shape
    assert data.shape[1:] == (3, 32, 32), f"Invalid sample shape: {data.shape[1:]}"
    break  # Just check first batch

Batch shape: torch.Size([10, 3, 32, 32]) torch.Size([10, 10])


In [8]:
# CNN Model
class Convolution_model(nn.Module):
    def __init__(self, input_size=(3,32,32)):
        super(Convolution_model, self).__init__()
        self.input_size = input_size
        self.conv_layer_01 = nn.Conv2d(in_channels=3 , out_channels=32, kernel_size=3, stride=1, padding=0)
        self.pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_layer_02 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)

        self.flattened_size = self.convolution_output_size()
        self.fc1 = nn.Linear(self.flattened_size , 64)
        self.fc2 = nn.Linear(64, 10)
        self.conv_activation = nn.ReLU()
        self.activation = nn.Sigmoid()

    def convolution_output_size(self):
        with torch.no_grad():
          dummy_target = torch.randn(1, *self.input_size)
          output = self.conv_layer_01(dummy_target)
          output = self.pool_layer(output)
          output = self.conv_layer_02(output)
          output = self.pool_layer(output)
        return output.view(1,-1).size(1)

    def forward(self, img_data):

        if img_data.shape[1:] != self.input_size:
            raise ValueError(f"Expected shape: {self.input_size} got {img_data.shape}")

        conv_layer_01 = self.conv_activation(self.conv_layer_01(img_data))
        pooled_01 = self.pool_layer(conv_layer_01)
        conv_layer_02 = self.conv_activation(self.conv_layer_02(pooled_01))
        pooled_02 = self.pool_layer(conv_layer_02)
        dense_input = pooled_02.view(pooled_02.size(0),-1)
        dense_01 = self.activation(self.fc1(dense_input))
        prob_values = torch.softmax(self.fc2(dense_01), dim=1)
        return prob_values


def train_model(model,train_datasets,learning_rate,num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)

    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_loss = 0

        for data,target in train_datasets:
            data,target = data.float() , target.float()
            optimizer.zero_grad()
            output = model.forward(data)
            loss_value = criterion(output,target)
            loss_value.backward()
            optimizer.step()
            total_loss +=loss_value.item()
        
        avg_loss = total_loss / len(train_datasets)    

        if epoch % 10 == 0:
            print(f"Epoch : {epoch}/{num_epochs} and Loss value: {avg_loss:.5f}")

    print(f"Training completed with total epochs: {num_epochs}")


In [9]:
conv_model = Convolution_model()

print(f"The flattened layer after whole convolution:- {conv_model.convolution_output_size()}")


The flattened layer after whole convolution:- 2304


In [10]:
karpathy_constant = 3e-4

train_model(model = conv_model , train_datasets = train_data_loader , learning_rate = karpathy_constant , num_epochs = 100)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch : 0/100 and Loss value: 2.19670
Epoch : 10/100 and Loss value: 1.97756
Epoch : 20/100 and Loss value: 1.89396
Epoch : 30/100 and Loss value: 1.82667
Epoch : 40/100 and Loss value: 1.77208
Epoch : 50/100 and Loss value: 1.73476
Epoch : 60/100 and Loss value: 1.71068
Epoch : 70/100 and Loss value: 1.69497
Epoch : 80/100 and Loss value: 1.68240
Epoch : 90/100 and Loss value: 1.67468
Training completed with total epochs: 100


In [11]:
# Model Serialization:- Saving up the model parameters after training phase

torch.save(conv_model.state_dict(), 'parameters_saved.pth')
parameters = torch.load('parameters_saved.pth' , map_location = 'cpu')
print(f"Model parameters loaded from file: \n{parameters.keys()}")

Model parameters loaded from file: 
odict_keys(['conv_layer_01.weight', 'conv_layer_01.bias', 'conv_layer_02.weight', 'conv_layer_02.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [12]:
new_model = Convolution_model()
new_model.load_state_dict(torch.load('parameters_saved.pth'))

<All keys matched successfully>

In [13]:
correct_predicted = 0
wrong_predicted = 0

for i,data in enumerate(x_test):
    data = data.reshape(1,3,32,32).float()
    predicted_value = torch.argmax(new_model.forward(data)).item()
    if predicted_value == y_test[i].item():
        correct_predicted += 1
    else:
        wrong_predicted += 1

print(f"Correctly predicted: {correct_predicted} and Wrongly predicted: {wrong_predicted}")
print(f"Total test data: {len(x_test)}")
print(f"Accuracy of the model: {correct_predicted / len(x_test) * 100:.2f}%")        
    

Correctly predicted: 4812 and Wrongly predicted: 5188
Total test data: 10000
Accuracy of the model: 48.12%
